# ESTADÍSTICAS CORRESPONDIENTES AL ARCHIVO DE ***IMÁGENES CLASIFICADAS***

En este archivo se registraron las imágenes que se fueron clasificando, con la información de cada una.

Primero mostraremos un gráfico de torta que muestre los porcentajes según el tipo de imagen.

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from unlpimage.paths import TAGS_FILE

df = pd.read_csv(TAGS_FILE, encoding='utf-8') # cargamos el archivo CSV

## Porcentaje tipos
Gráfico que muestra el porcentaje de tipos de imágenes tagueadas.

In [2]:
"""
Calculamos los porcentajes a mostrar, extrayendo la informarcion del dataframe donde se cargó la información contenida
en el archivo de tags obteniendo el porcentaje de veces que aparece cada valor en la columna 'tipo'.
- con df[tipo] accedemos a la columna 'tipo' del DataFrame
- .value_counts() -> utilizamos el método value_counts() para contar la cantidad de veces que aparece cada valor único en la columna 'tipo'.
Esto genera una serie que contiene los valores únicos como índices y la cantidad de veces que apareció cada valor como valores.
- normalize=True -> indica que se deben calcular las proporciones en lugar de las frecuencias absolutas.
Esto normaliza los conteos dividiéndolos por el número total de observaciones, lo que da como resultado los porcentajes relativos.
"""
porcentajes = df['tipo'].value_counts(normalize=True) * 100

plt.figure(figsize=(8, 6)) # creo una nueva figura de 8x6
porcentajes.plot(kind='pie', autopct='%1.1f%%') # creo el gráfico de torta, mostrando los porcentajes
plt.title('Tipos de imágenes') # inserto el título
plt.ylabel('') # oculta la etiqueta del eje Y en el gráfico
plt.legend() # agrega la descripción de los elementos del gráfico
plt.show() # mostramos el gráfico

## Máximo ancho y alto
Calculamos los valores máximos de ancho y de alto de las imágenes clasificadas.

In [3]:
"""
Extraemos los valores de ancho y alto separando el valor contenido en la columna "resolucion"
por la 'x', que nos permite obtener el ancho y el alto y mediante el expand=True asignamos los
resultados a dos nuevas columnas del DataFrame llamadas "ancho" y "alto".
"""

df[['ancho', 'alto']] = df['resolucion'].str.split('x', expand=True)

# convertimos las columnas de ancho y alto a numérico
df['ancho'] = pd.to_numeric(df['ancho'])
df['alto'] = pd.to_numeric(df['alto'])

# calculamos los valores máximos de ancho y alto
ancho_maximo = df['ancho'].max()
alto_maximo = df['alto'].max()

print("Valor máximo de ancho:", ancho_maximo)
print("Valor máximo de alto:", alto_maximo)

Valor máximo de ancho: 768
Valor máximo de alto: 768


## Ancho y alto
Generamos un gráfico de dispersión para visualizar la relación entre el ancho y el alto de las imágenes.

In [4]:
"""
Por si no se ejecutó el codigo anterior volvemos a repetir el proceso de creación de las 2 nuevas columnnas
del DataFrame llamadas "ancho" y "alto" mediante el expand=True.
"""
df[['ancho', 'alto']] = df['resolucion'].str.split('x', expand=True)

# convertimos las columnas de ancho y alto a numérico
df['ancho'] = pd.to_numeric(df['ancho'])
df['alto'] = pd.to_numeric(df['alto'])

plt.scatter(df['ancho'], df['alto']) # generamos el gráfico de dispersión
plt.xlabel('Ancho') # configuramos la etiqueta del eje x
plt.ylabel('Alto') # configuramos la etiqueta del eje y
plt.title('Relación entre Ancho y Alto de las imágenes') # configuramos título
plt.show()

## Cantidad cambios x día
En base a la fecha de última actualización, graficamos la cantidad de cambios realizados para cada día de la semana.

In [5]:
"""
Convertimos el timestamp de segundos a tipo datetime. Luego obtenemos el día de
la semana para cada fecha de última actualización mediante el dt.dayofweek -> creando la nueva columna del
DataFrame llamada "dia_semana".
"""

df['fechaActualizacion'] = pd.to_datetime(df['fechaActualizacion'], unit='s')
df['dia_semana'] = df['fechaActualizacion'].dt.dayofweek 

cambios_por_dia = df['dia_semana'].value_counts() # calculamos la cantidad de cambios realizados para cada día de la semana
dias_semana_ordenados = [0, 1, 2, 3, 4, 5, 6] # ordenamos los días de la semana en orden cronológico

# verificamos si falta algún día y si es asi, establecemos su cantidad en 0
for dia in dias_semana_ordenados:
    if dia not in cambios_por_dia.index:
        cambios_por_dia[dia] = 0

cambios_por_dia = cambios_por_dia.reindex(dias_semana_ordenados) # se reordenan los días de la semana en orden cronológico

plt.bar(range(len(cambios_por_dia)), cambios_por_dia.values) # generamos el gráfico de barras

plt.xlabel('Día de la semana') # configuramos la etiqueta del eje x
plt.ylabel('Cantidad de cambios') # configuramos la etiqueta del eje y
plt.title('Cantidad de cambios realizados por día de la semana') # configuramos título
plt.xticks(range(len(cambios_por_dia)), ['L', 'M', 'M', 'J', 'V', 'S', 'D']) # cambiamos las etiquetas del eje x
plt.yticks(range(1, int(max(cambios_por_dia.values))+1)) # ajustamos los límites del eje y
plt.show()

## Actualizaciones
Creamos un gráfico de líneas para visualizar la evolución de la cantidad de actualizaciones a lo largo del tiempo.

In [6]:
from matplotlib.dates import DateFormatter, AutoDateLocator

"""
Convertimos la columna 'fechaActualizacion' del DataFrame df a tipo datetime utilizando pd.to_datetime().
Se eusa unit='s' para indicar que los valores en la columna representan una marca de tiempo en segundos.
"""
f = pd.to_datetime(df['fechaActualizacion'], unit='s')

# truncamos las fechas al día más cercano
f = f.dt.floor('D')

# contamos la cantidad de actualizaciones por fecha
actualizaciones_por_fecha = f.value_counts().sort_index()

# generamos el gráfico
plt.plot(actualizaciones_por_fecha.index, actualizaciones_por_fecha.values, drawstyle='default')#, drawstyle='steps-post')

# configuramos etiquetas y título
plt.xlabel('Fecha de actualización') # configuramos la etiqueta del eje x
plt.ylabel('Cantidad de actualizaciones') # configuramos la etiqueta del eje y
plt.title('Evolución de las actualizaciones') # configuramos título

# configuramos el comportamiento del eje y
plt.ylim(bottom=0)  # fijamos límite inferior en 0
plt.yticks(range(0, max(actualizaciones_por_fecha.values) + 2)) # establecemos valores enteros en el eje y

# le damos formato las fechas en el eje x
date_formatter = DateFormatter('%d-%m')
plt.gca().xaxis.set_major_formatter(date_formatter)
plt.gca().xaxis.set_major_locator(AutoDateLocator())
plt.show() # mostramos el gráfico

## Nube de tags
Con la lista de tags generamos una nube de palabras

In [7]:
"""
Obtenemos la lista de tags como una cadena de texto, sacando todo lo que nos impida un buen uso de los tags.
- con df[´tags´] accedemos a la columna 'tags' del DataFrame
- .str.replace("[\[\]']", '', regex=True) -> utilizamos el método replace de la clase str para
eliminar dichos caracteres. Se utiliza regex=True para que la búsqueda y el reemplazo se realicen utilizando expresiones regulares.
- .str.cat(sep=',') -> utilizamos el método cat de la clase str para concatenar todos los valores de la columna 'tags' en una sola cadena.
El parámetro sep=',' indica que se debe utilizar una coma como separador entre los valores concatenados.
"""

tags_text = df['tags'].str.replace("[\[\]']", '', regex=True).str.cat(sep=' ')

# creamos la nube de palabras usando wordcloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(tags_text)

plt.figure(figsize=(10, 6)) # creamos la nueva figura con su tamaño de 10x6
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off') # ocultamos los ejes x e y
plt.show()

## 3 tags más usados
Informamos cuáles son los 3 tags más utilizados.

In [12]:
"""
Obtenemos la lista de tags como una cadena de texto, sacando todo lo que nos impida un buen uso de los tags:
- con df[´tags´] accedemos a la columna 'tags' del DataFrame
- .str.replace("[\[\]']", '', regex=True) -> utilizamos el método replace de la clase str para
eliminar dichos caracteres. Se utiliza regex=True para que la búsqueda y el reemplazo se realicen utilizando expresiones regulares.
- .str.cat(sep=',') -> utilizamos el método cat de la clase str para concatenar todos los valores de la columna 'tags' en una sola cadena.
El parámetro sep=',' indica que se debe utilizar una coma como separador entre los valores concatenados.
"""

tags_text = df['tags'].str.replace("[\[\]']", '', regex=True).str.cat(sep=',')

tags_list = tags_text.split(',') # dividimos la cadena de texto en tags individuales

tag_counts = pd.Series(tags_list).value_counts() # contamos la cantidad de veces que aparece cada tag
top_tags = tag_counts.head(3).reset_index() # obtenemos los 3 tags más utilizados
top_tags.columns = ['Tag', 'Usos'] # renombramos las columnas
top_tags.index = top_tags.index + 1 # agregamos el número de índice a cada tag (para que empiece desde 1 y no desde 0)
display(top_tags) # mostramos el DataFrame

Tag  Usos
1   Plantilla     4
2      Hombre     3
3       Negro     2

## Tamaño promedio tags
Calculamos el tamaño en bytes promedio de las imágenes actualizadas por cada perfil,
incluyendo los perfiles que no hayan realizado actualizaciones.

In [15]:
"""
Calculamos el tamaño en megabytes promedio de las imágenes actualizadas por cada perfil:
- df.groupby('ultimoPerfil') -> agrupamos el DataFrame df por los valores únicos en la columna 'ultimoPerfil'.
Esto crea un objeto GroupBy que se puede utilizar para aplicar operaciones en cada grupo individualmente.
- ['tamaño(mb)'] -> seleccionamos la columna 'tamaño(mb)' del DataFrame.
- .mean() -> calcula el promedio del tamaño en megabytes para cada grupo.

Leemos el archivo de perfiles para obtener una lista de los nicks.
Con esa lista obtenemos una nueva lista de aquellos perfiles que no taguearon ninguna imagen.
Creamos un nuevo Dataframe con dicha lista estableciendo sus tamaños promedios en 0.
Y por último concatenamos ambos DataFrames en uno solo.
"""

from unlpimage.read_data import read_profiles
profiles_list = read_profiles()
profiles_list = [profile['nick'] for profile in profiles_list['profiles']]

perfil_size_avg = df.groupby('ultimoPerfil')['tamaño(mb)'].mean() 

perfiles_unicos = df['ultimoPerfil'].unique() # obtenemos la lista de perfiles únicos

resultado = pd.DataFrame({'Perfil': perfil_size_avg.index, 'Tamaño Promedio (MB)': perfil_size_avg.values}) #se crea el dataframa con los nicks y el tamaño promedio

# encontramos perfiles faltantes
perfiles_faltantes = list(set(profiles_list) - set(perfiles_unicos))

# creamos DataFrame para perfiles faltantes con tamaño promedio de 0
df_faltantes = pd.DataFrame({'Perfil': perfiles_faltantes, 'Tamaño Promedio (MB)': 0}) 

# concatenamos DataFrame de perfiles faltantes con el resultado existente
resultado = pd.concat([resultado, df_faltantes], ignore_index=True)

resultado.index = resultado.index + 1 # agregamos el número de índice a cada nick (para que empiece desde 1 y no desde 0)

display(resultado) # mostramos el dataframe

#tener en cuenta que NO se pide mostrar ordenado de mayor a menor

Perfil  Tamaño Promedio (MB)
1    bauti              0.097117
2  esteban              0.083823
3     nico              0.096236
4    tucci              0.237335
5    messi              0.000000